In [56]:
import os
import os.path as osp
from PIL import Image
import numpy as np
from glob import glob
from tqdm import tqdm

In [60]:
gt_dir = "/home/user/WEAK_DA/weak-da-seg/datasets/cityscapes/gtFine/val"
infer_dir = "/home/user/WEAK_DA/weak-da-seg/result/gta5-cityscapes-weakimage-cw-pa-official"

In [6]:
infer_list = glob(osp.join(infer_dir, '*.png'))

tp = 0
tpfp = 0
tpfn = 0

for infer_path in tqdm(infer_list):
    infer = Image.open(infer_path)
    infer_arr = np.array(infer)

    base_name = osp.basename(infer_path)
    base_name = base_name.replace('leftImg8bit_color', 'gtFine_labelIds')

    gt_path = osp.join(gt_dir, base_name)
    gt = Image.open(gt_path)
    gt_arr = np.array(gt)

    mul = np.sqrt(np.multiply(gt_arr, infer_arr))

    tp_img = len(mul[mul==1])
    tpfp_img = len(infer_arr[infer_arr==1])
    tpfn_img = len(gt_arr[gt_arr==1])

    if tpfp_img == 0:
        pass
        # print('tpfp_img is zero')
    else:
        pre_img = tp_img / tpfp_img
    
    if tpfn_img == 0:
        pass
        # print('tpfn_img is zero')
    else:
        recall_img = tp_img / tpfn_img

    if tpfp_img !=0 and tpfn_img != 0:
        if (pre_img + recall_img) != 0:
            f1_img = (2 * pre_img * recall_img) / (pre_img + recall_img)
            # print(f"F1-score of this image is {f1_img}")

    tp = tp + tp_img
    tpfp = tpfp + tpfp_img
    tpfn = tpfn + tpfn_img
    
pre = tp / tpfp  # 오탐지
recall = tp / tpfn  # 미탐지

f1 = (2 * pre * recall) / (pre + recall)

print('The precision of crack is ', pre*100, '%')
print('The recall of crack is ', recall*100, '%')
print('The f1-score of crack is ', f1*100, '%')

In [61]:
# cityscapes
import json

with open('/home/user/WEAK_DA/weak-da-seg/data/cityscapes_list/info.json', 'r') as fp:
    info = json.load(fp)
mapping = np.array(info['label2train'], dtype=np.int)

def label_mapping(input, mapping):
    output = np.copy(input)
    for ind in range(len(mapping)):
        output[input == mapping[ind][0]] = mapping[ind][1]
    return np.array(output, dtype=np.int64)

infer_list = glob(osp.join(infer_dir, '*.png'))

num_classes = 19

tp_list = np.zeros(num_classes, dtype=np.int32)
tpfp_list = np.zeros(num_classes, dtype=np.int32)
tpfn_list = np.zeros(num_classes, dtype=np.int32)

for infer_path in tqdm(infer_list):
    infer = Image.open(infer_path)
    infer_arr = np.array(infer)

    base_name = osp.basename(infer_path)
    city = base_name.split('_')[0]
    base_name = base_name.replace('_leftImg8bit_color', '_gtFine_labelIds')
    base_name = city + '/' + base_name

    gt_path = osp.join(gt_dir, base_name)
    gt = Image.open(gt_path)
    gt_arr = np.array(gt)

    gt_arr = label_mapping(gt_arr, mapping)

    # mul = np.sqrt(np.multiply(gt_arr, infer_arr))

    for i in range(num_classes):
        tp_img = np.sum((infer_arr==i) & (gt_arr==i))
        tpfp_img = len(infer_arr[infer_arr==i])
        tpfn_img = len(gt_arr[gt_arr==i])

        if tpfp_img == 0:
            pass
        else:
            pre_img = tp_img / tpfp_img

        if tpfn_img == 0:
            pass
        else:
            recall_img = tp_img / tpfn_img

        if tpfp_img !=0 and tpfn_img != 0:
            if (pre_img + recall_img) != 0:
                f1_img = (2 * pre_img * recall_img) / (pre_img + recall_img)

        tp_list[i] += tp_img
        tpfp_list[i] += tpfp_img
        tpfn_list[i] += tpfn_img
    
precision_list = tp_list / tpfp_list
recall_list = tp_list / tpfn_list
f1_list = (2 * precision_list * recall_list) / (precision_list + recall_list)

print('precision_list:', precision_list*100)
print('recall_list:', recall_list*100)
print('f1_list:', f1_list*100)

/home/user/anaconda3/envs/mmseg2/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
100%|██████████| 500/500 [01:04<00:00,  7.81it/s]

precision_list: [81.40804798 63.97438598 79.10307877 27.3142755  46.43174262 54.97433163
 67.45374116 76.51017443 85.16595436 41.02632251 82.7391576  68.17451764
 46.87468372 71.19646432 49.61342761 78.30056057 91.48096988 34.60969181
 74.56664082]
recall_list: [95.14172965 60.63973658 92.76126066 48.26406621 45.68922275 47.63880971
 50.2551713  41.66306325 93.36006404 57.0074384  96.62900275 74.25507562
 53.45024995 93.13799516 66.27467973 60.12014751 35.49989919 60.10693501
 51.99185176]
f1_list: [87.74072215 62.262444   85.38945697 34.88560271 46.05749023 51.04437286
 57.5980059  53.94873321 89.07495976 47.71428805 89.14628184 71.08500202
 49.94697668 80.70244026 56.7463582  68.01643073 51.15046435 43.9264481
 61.26586461]


In [53]:
np.mean(f1_list)*100

61.124387168270836

In [90]:
# CRACK500_20160222_080850_641_361_leftImg8bit_quarter_1_color.png
# CRACK500_20160222_080850_641_361_gtFine_labelIds_quarter_1.png

infer_list = glob(osp.join(infer_dir, 'CRACK500*'))

tp = 0
tpfp = 0
tpfn = 0

for infer_path in tqdm(infer_list):
    infer = Image.open(infer_path)
    infer_arr = np.array(infer)

    base_name = osp.basename(infer_path)
    base_name = base_name.replace('_leftImg8bit_', '_gtFine_labelIds_')
    base_name = base_name.replace('_color.png', '.png')

    gt_path = osp.join(gt_dir, base_name)
    gt = Image.open(gt_path)
    gt_arr = np.array(gt)

    mul = np.sqrt(np.multiply(gt_arr, infer_arr))

    tp_img = len(mul[mul==1])
    tpfp_img = len(infer_arr[infer_arr==1])
    tpfn_img = len(gt_arr[gt_arr==1])

    if tpfp_img == 0:
        pass
        # print('tpfp_img is zero')
    else:
        pre_img = tp_img / tpfp_img
    
    if tpfn_img == 0:
        pass
        # print('tpfn_img is zero')
    else:
        recall_img = tp_img / tpfn_img

    if tpfp_img !=0 and tpfn_img != 0:
        if (pre_img + recall_img) != 0:
            f1_img = (2 * pre_img * recall_img) / (pre_img + recall_img)
            # print(f"F1-score of this image is {f1_img}")

    tp = tp + tp_img
    tpfp = tpfp + tpfp_img
    tpfn = tpfn + tpfn_img
    
pre = tp / tpfp  # 오탐지
recall = tp / tpfn  # 미탐지

f1 = (2 * pre * recall) / (pre + recall)

print('The precision of crack is ', pre*100, '%')
print('The recall of crack is ', recall*100, '%')
print('The f1-score of crack is ', f1*100, '%')

100%|██████████| 2320/2320 [00:11<00:00, 206.74it/s]

The precision of crack is  31.131888645913087 %
The recall of crack is  20.58615546667772 %
The f1-score of crack is  24.783841331696514 %
